# Nå skal vi gjøre noe bra her

Dette funket ikke så bra... prøver på noe nytt...

### Imports

In [17]:
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import DistributionLoss
from sklearn.metrics import mean_absolute_error

In [18]:
# gather data set and split in x and y
def read_dataset(filename):
    df = pd.read_csv(filename)
    df = df.loc[:, ~df.columns.str.match('Unnamed')]
    df = df.replace('DC', 1)
    df = df.replace('LTE', 0)
    return df

def vertical_handover(y):
    new_y = []
    for seq in y:
        if 0 in seq and 1 in seq:
            new_y.append(1)
        else:
            new_y.append(0)
    return np.array(new_y)

def accuracy(y_pred, y_true):
    return np.mean(y_pred == y_true)

def precision_recall(y_pred, y_true):
    tp, fp, fn =  0, 0, 0

    # Loop through true and predicted labels to count TP, FP, and FN
    for yt, yp in zip(y_true, y_pred):
        if yt == 1 and yp == 1:
            tp += 1
        elif yt == 0 and yp == 1:
            fp += 1
        elif yt == 1 and yp == 0:
            fn += 1
    print(tp, fp, fn)

    # Precision calculation
    if tp + fp == 0:
        precision = 0
    else:
        precision = tp / (tp + fp)

    # Recall calculation
    if tp + fn == 0:
        recall = 0
    else:
        recall = tp / (tp + fn)

    return precision, recall

In [19]:
df = read_dataset('datasets/Op1_merged.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.fillna(0)

df = df.rename(columns={'Timestamp': 'ds', 'Mode': 'y', 'campaign': 'unique_id'})

test_ids = [
    'Capacity_Ookla_4G5G_TIM_Driving_Tuscolana_Stat_Appia_No_Flight_Mode',
    'Ping_UNIDATA_4G5G_TIM_Walking_DIET_to_Termini',
    'Ping_UNIDATA_4G5G_TIM_Driving_Viale_Marconi'
]

train = df.loc[~df['unique_id'].isin(test_ids)]
test = df.loc[df['unique_id'].isin(test_ids)]

In [20]:
models = [LSTM(h=10,
                   loss=DistributionLoss(distribution='Normal', level=[90]),
                   max_steps=1000,
                   encoder_n_layers=10,
                   encoder_hidden_size=300,
                   context_size=10,
                   decoder_hidden_size=300,
                   decoder_layers=2,
                   learning_rate=1e-3,
                   scaler_type='standard')]

model = NeuralForecast(models=models, freq='D')
model.fit(train)
#p = p.merge(test[['ds', 'unique_id', 'y']], on=['ds', 'unique_id'], how='left')


Seed set to 1
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | loss            | DistributionLoss | 3      | train
1 | padder          | ConstantPad1d    | 0      | train
2 | scaler          | TemporalNorm     | 0      | train
3 | hist_encoder    | LSTM             | 6.9 M  | train
4 | context_adapter | Linear           | 30.1 K | train
5 | mlp_decoder     | MLP              | 3.9 K  | train
-------------------------------------------------------------
6.9 M     Trainable params
3         Non-trainable params
6.9 M     Total params
27.597    Total estimated model params size (MB)
11        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [16]:
y_pred = model.predict(test)
y_pred = y_pred.merge(test[['ds', 'unique_id', 'y']], on=['ds', 'unique_id'], how='left')
y_pred = (y_pred['y'] > 0.5).astype(int)


test_bulk = vertical_handover(test['y'])
y_pred_bulk = vertical_handover(y_pred['y'])
acc = accuracy(test_bulk, y_pred_bulk)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

/Users/ihneskarbovik/Library/Python/3.9/lib/python/site-packages/neuralforecast/core.py:210: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(


TypeError: argument of type 'int' is not iterable